In [1]:
from ffiec_data_connect import methods, credentials, ffiec_connection
import pandas as pd
creds = credentials.WebserviceCredentials(username="your-username", password="your-token")

conn = ffiec_connection.FFIECConnection()

In [13]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col
import pandas as pd

In [27]:
connection_parameters = {
"user": "your-username",
"password": "your-password",
"account": "your-account",
"role": "your-role",
"warehouse": "your-warehouse",
"database": "your-database",
"schema": "your-schema" }
session = snowpark.session.Session.builder.configs(connection_parameters).create()

In [25]:
session.close()

In [15]:
df_schema = session.sql("DESCRIBE TABLE SCHEDULE_ENT;").collect()
schema_cols = pd.DataFrame(df_schema)
schema_cols['name']

0    RCON9224
1    RCON9999
2    RSSD9017
3    RSSD9130
4    RSSD9200
5    RSSD9220
6      IDRSSD
Name: name, dtype: object

In [16]:
def grab_id_data(id_rssd_list,reporting_period):
    skipper = 0
    for id_rssd in id_rssd_list:
        df = methods.collect_data(
            session=conn,
            creds=creds,
            rssd_id=id_rssd,
            reporting_period=reporting_period,
            series="call",
            output_type='pandas'
        )

        result_df = pd.DataFrame(columns=df['mdrm'].unique())
        for index, row in df.iterrows():
            mdrm = row['mdrm']
            data_type = row['data_type']
            if data_type == 'str':
                value = row['str_data']
            elif data_type == 'bool':
                value = row['bool_data']
            elif data_type == 'float':
                value = row['float_data']
            elif data_type == 'int':
                value = row['int_data']
            else:
                value = None
            result_df.at[0, mdrm] = value
        result_df.insert(0, 'IDRSSD',id_rssd)
        result_df['RCON9999']=pd.to_datetime(result_df['RCON9999'], format='%Y%m%d.%f').dt.strftime('%Y-%m-%d')
        date_column = result_df.pop('RCON9999') 
        result_df.insert(1, 'RCON9999', date_column) 
        if skipper == 0:
            final_df = result_df
            skipper+=1
        else:
            final_df = pd.concat([final_df,result_df]).reset_index(drop=True)
    return final_df

In [25]:
df = grab_id_data([37,242,279,354,5805479,5859511],'09/30/2023')

,IDRSSD,RCON9999,RCONC752,RCONF164,RCONG322,RCON1460,RIADGW44,RIADB522,RCONC027,RCONC026,...,RCONJ258,RCONJ257,RCONJ256,RCONJ255,RCONJJ31,RIADJ316,RIADJ315,RCOA3792,RCONB882,RCONC001
0,37,2023-09-30,FFIEC051,0.0,0.0,0.0,38.0,955.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,242,2023-09-30,FFIEC051,19.0,0.0,0.0,49.0,178.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,279,2023-09-30,FFIEC051,13393.0,28627.0,8035.0,1849.0,1755.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,354,2023-09-30,FFIEC051,1354.0,0.0,657.0,348.0,317.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5805479,2023-09-30,FFIEC051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5859511,2023-09-30,FFIEC051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,118.0,163590.0,626091.0,0.0,468.0,23099.0,151504.0,18089.0,0.0


In [29]:
df = session.create_dataframe([[1, 2], [3, 4]], schema=["a", "b"])
df.write.mode("overwrite").save_as_table("saved_table", table_type="temporary")
session.table("saved_table").show()


stage_created_result = session.sql("create temporary stage if not exists my_stage").collect()
df.write.copy_into_location("@my_stage/copied_from_dataframe.csv")  # default CSV

-------------
|"A"  |"B"  |
-------------
|1    |2    |
|3    |4    |
-------------



[Row(rows_unloaded=2, input_bytes=8, output_bytes=28)]

In [ ]:
def main(session: snowpark.Session):
    df_table = session.table("sample_product_data")

In [38]:
def match_data_type(id_rssd_set, mdrm_set):
    master_data_type_list = []
    trials = 0
    while len(mdrm_set) != 0 and trials != len(id_rssd_set):
        for id_rssd in id_rssd_set:
                print(id_rssd)
                time_series = methods.collect_data(
                session=conn,
                creds=creds,
                rssd_id=id_rssd,
                reporting_period="9/30/2023",
                series="call",
                output_type='pandas'
                )
                print(time_series)
                for mdrm in mdrm_set.copy():
                    if mdrm in time_series['mdrm'].values:
                        print('made it')
                        data_type = time_series['data_type'][time_series['mdrm']==mdrm].values[0]
                        pair_mdrm_type = (mdrm,data_type)
                        master_data_type_list.append(pair_mdrm_type)
                        mdrm_set.remove(mdrm)
                        print(mdrm_set,len(mdrm_set))
                trials +=1
                if not mdrm_set:
                    break
                #time.sleep(2)
    return master_data_type_list

In [39]:
id_rssd_set = set([37,279])
mdrm_set = set(['RCONC752','RCONF164','RCOALB61','RIADC235','RIADGW44'])
ml_ret = match_data_type(id_rssd_set,mdrm_set)
ml_ret

37
         mdrm rssd    quarter  int_data  float_data bool_data  str_data  \
0    RCONC752   37  9/30/2023       NaN         NaN      None  FFIEC051   
1    RCONF164   37  9/30/2023       0.0         NaN      None      None   
2    RCONG322   37  9/30/2023       0.0         NaN      None      None   
3    RCON1460   37  9/30/2023       0.0         NaN      None      None   
4    RIADGW44   37  9/30/2023      38.0         NaN      None      None   
..        ...  ...        ...       ...         ...       ...       ...   
578  RIADC232   37  9/30/2023       0.0         NaN      None      None   
579  RCOALB60   37  9/30/2023       0.0         NaN      None      None   
580  RCOALB61   37  9/30/2023       0.0         NaN      None      None   
581  RIADC235   37  9/30/2023       0.0         NaN      None      None   
582  RIADC234   37  9/30/2023       1.0         NaN      None      None   

    data_type  
0         str  
1         int  
2         int  
3         int  
4         int  


[('RCOALB61', 'int'),
 ('RCONF164', 'int'),
 ('RIADGW44', 'int'),
 ('RCONC752', 'str'),
 ('RIADC235', 'int')]

In [14]:
for mdrm in mdrm_set:
    if mdrm in time_series['mdrm'].values:
        print(f"{mdrm} found!")
    else:
        print(f"{mdrm} not found.")

RCOALB61 found!
RCONC752 found!
RIADC235 found!
RCONF164 found!
RIADGW44 found!


In [13]:

time_series['data_type'][time_series['mdrm']==mdrm]

4    int
Name: data_type, dtype: object

In [2]:
time_series = methods.collect_data(
    session=conn,
    creds=creds,
    rssd_id="37",
    reporting_period="9/30/2023",
    series="call",
    output_type='pandas'
)
time_series

,mdrm,rssd,quarter,int_data,float_data,bool_data,str_data,data_type
0,RCONC752,37,9/30/2023,NaN,NaN,None,FFIEC051,str
1,RCONF164,37,9/30/2023,0.0,NaN,None,None,int
2,RCONG322,37,9/30/2023,0.0,NaN,None,None,int
3,RCON1460,37,9/30/2023,0.0,NaN,None,None,int
4,RIADGW44,37,9/30/2023,38.0,NaN,None,None,int
...,...,...,...,...,...,...,...,...
578,RIADC232,37,9/30/2023,0.0,NaN,None,None,int
579,RCOALB60,37,9/30/2023,0.0,NaN,None,None,int
580,RCOALB61,37,9/30/2023,0.0,NaN,None,None,int
581,RIADC235,37,9/30/2023,0.0,NaN,None,None,int


In [4]:
df= time_series

In [14]:
result_df = pd.DataFrame(columns=df['mdrm'].unique())

# Iterate through each row in the original DataFrame
for index, row in df.iterrows():
    mdrm = row['mdrm']
    data_type = row['data_type']

    # Depending on the 'data_type', select the corresponding value
    if data_type == 'str':
        value = row['str_data']
    elif data_type == 'bool':
        value = row['bool_data']
    elif data_type == 'float':
        value = row['float_data']
    elif data_type == 'int':
        value = row['int_data']
    else:
        value = None

    # Set the value in the result DataFrame
    result_df.at[0, mdrm] = value

# Transpose the result DataFrame to have one row and 583 columns
#result_df = result_df.T

# Reset the index and rename columns
#result_df = result_df.reset_index()
#result_df.columns = ['mdrm', 0]

# Display the resulting DataFrame
print(result_df)

   RCONC752 RCONF164 RCONG322 RCON1460 RIADGW44 RIADB522 RCONC027 RCONC026  \
0  FFIEC051      0.0      0.0      0.0     38.0    955.0      0.0      0.0   

  RCONHK09 RCONHK17  ... RIADC895 RIADC894 RCONG472 RIADC896 RIADC233  \
0   5162.0    964.0  ...      0.0      0.0      0.0      0.0      0.0   

  RIADC232 RCOALB60 RCOALB61 RIADC235 RIADC234  
0      0.0      0.0      0.0      0.0      1.0  

[1 rows x 583 columns]


In [15]:
result_df['RCON9999']# = '09/30/2023'

0    20230930.0
Name: RCON9999, dtype: object

In [18]:
result_df['IDRSSD'] = 37

In [19]:
result_df['IDRSSD']

0    37
Name: IDRSSD, dtype: int64

In [16]:
result_df['RCON9999']=pd.to_datetime(result_df['RCON9999'], format='%Y%m%d.%f').dt.strftime('%Y-%m-%d')

0    2023-09-30
Name: RCON9999, dtype: object

In [2]:
last_filing_date_time = methods.collect_filers_submission_date_time(
    session=conn,
    creds=creds,
    since_date="10/31/2022",
    reporting_period="9/30/2023",
)

In [6]:
df =pd.DataFrame(last_filing_date_time)
df

,rssd,datetime
0,175458,10/1/2023 8:26:23 AM
1,723158,10/1/2023 2:00:50 PM
2,359744,10/2/2023 9:15:07 AM
3,688556,10/2/2023 11:38:07 AM
4,199137,10/2/2023 2:27:02 PM
...,...,...
4619,856869,10/31/2023 12:26:27 AM
4620,543459,10/31/2023 12:30:06 AM
4621,402846,10/31/2023 12:33:03 AM
4622,666554,10/31/2023 12:59:07 AM


In [3]:
filers = methods.collect_filers_on_reporting_period(
    session=conn,
    creds=creds,
    reporting_period="9/30/2023",
    output_type="list"
)

In [ ]:
[37,242,279,354,5805479]

In [3]:
df2 = pd.DataFrame(filers)
df2

,id_rssd,fdic_cert_number,occ_chart_number,ots_dock_number,primary_aba_rout_number,name,state,city,address,filing_type,has_filed_for_reporting_period
0,37,10057,None,16553,61107146,BANK OF HANCOCK COUNTY,GA,SPARTA,12855 BROAD STREET,051,True
1,242,3850,None,None,81220537,FIRST COMMUNITY BANK XENIA-FLORA,IL,XENIA,260 FRONT STREET,051,True
2,279,28868,None,2523,311972526,"MINEOLA COMMUNITY BANK, SSB",TX,MINEOLA,215 W BROAD,051,True
3,354,14083,None,None,101107475,BISON STATE BANK,KS,BISON,223 MAIN STREET,051,True
4,457,10202,None,None,91208332,LOWRY STATE BANK,MN,LOWRY,400 FLORENCE AVE.,051,True
...,...,...,...,...,...,...,...,...,...,...,...
4664,5805479,59346,25287,None,122402463,"BESSEMER TRUST COMPANY OF NEVADA, NATIONAL ASS...",NV,LAS VEGAS,1700 SOUTH PAVILION CENTER DRIVE SUITE...,051,True
4665,5805488,59349,25275,None,121202457,"INSPIRE TRUST COMPANY, NATIONAL ASSOCIATION",NV,RENO,241 RIDGE STREET SUITE 310,041,True
4666,5805817,59337,25237,None,91018470,CERIDIAN NATIONAL TRUST BANK,MN,BLOOMINGTON,3311 EAST OLD SHAKOPEE ROAD,051,True
4667,5859511,59344,25288,None,53012977,"TIAA TRUST, NATIONAL ASSOCIATION",NC,CHARLOTTE,8500 ANDREW CARNEGIE BOULEVARD,051,True


In [5]:
df3 = df2[df2['has_filed_for_reporting_period']==True]
df3

,id_rssd,fdic_cert_number,occ_chart_number,ots_dock_number,primary_aba_rout_number,name,state,city,address,filing_type,has_filed_for_reporting_period
0,37,10057,None,16553,61107146,BANK OF HANCOCK COUNTY,GA,SPARTA,12855 BROAD STREET,051,True
1,242,3850,None,None,81220537,FIRST COMMUNITY BANK XENIA-FLORA,IL,XENIA,260 FRONT STREET,051,True
2,279,28868,None,2523,311972526,"MINEOLA COMMUNITY BANK, SSB",TX,MINEOLA,215 W BROAD,051,True
3,354,14083,None,None,101107475,BISON STATE BANK,KS,BISON,223 MAIN STREET,051,True
4,457,10202,None,None,91208332,LOWRY STATE BANK,MN,LOWRY,400 FLORENCE AVE.,051,True
...,...,...,...,...,...,...,...,...,...,...,...
4664,5805479,59346,25287,None,122402463,"BESSEMER TRUST COMPANY OF NEVADA, NATIONAL ASS...",NV,LAS VEGAS,1700 SOUTH PAVILION CENTER DRIVE SUITE...,051,True
4665,5805488,59349,25275,None,121202457,"INSPIRE TRUST COMPANY, NATIONAL ASSOCIATION",NV,RENO,241 RIDGE STREET SUITE 310,041,True
4666,5805817,59337,25237,None,91018470,CERIDIAN NATIONAL TRUST BANK,MN,BLOOMINGTON,3311 EAST OLD SHAKOPEE ROAD,051,True
4667,5859511,59344,25288,None,53012977,"TIAA TRUST, NATIONAL ASSOCIATION",NC,CHARLOTTE,8500 ANDREW CARNEGIE BOULEVARD,051,True


In [ ]:
filers = methods.collect_filers_on_reporting_period(
    session=conn,
    creds=creds,
    reporting_period="9/30/2023",
    output_type="list"
)